<a href="https://colab.research.google.com/github/edabeyza/ensembleProject/blob/main/KollektifFinalProje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Tüm proje dosyalarını Drive'da sakla ve bunu Colab'a bağla
from google.colab import drive
drive.mount('/content/drive')

# Model eğitimi, veri işleme ve sonuç analizinde gereken kütüphaneleri yükle
!pip install -q datasets transformers accelerate peft bitsandbytes
!pip install -q pandas numpy matplotlib tqdm scikit-learn


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Hugging Face'ten Türkçe GSM8K veri setini indirip 'dataset' değişkenine yükle
from datasets import load_dataset

dataset = load_dataset("ytu-ce-cosmos/gsm8k_tr")
dataset


DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 8792
    })
})

In [8]:
# Train/test ayrımı ve 3 iterasyona bölme

full_data = dataset["train"].shuffle(seed=42)

# 1500 soru: süreç için (3 iterasyon * 500)
train_1500 = full_data.select(range(0, 1500))

# 500 soru: sabit test seti
test_500   = full_data.select(range(1500, 2000))

len(train_1500), len(test_500)


(1500, 500)

In [9]:
# 1500 süreci 3 iterasyona böl (her biri 500 soru)
iter1 = train_1500.select(range(0, 500))
iter2 = train_1500.select(range(500, 1000))
iter3 = train_1500.select(range(1000, 1500))

len(iter1), len(iter2), len(iter3)


(500, 500, 500)

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Kullanacağım model
MODEL_NAME = "ytu-ce-cosmos/Turkish-Gemma-9b-T1"

# Tokenizer ve model yükleme
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.eval()

model


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/853 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/223 [00:00<?, ?B/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RM

In [22]:
import re
from tqdm import tqdm
import torch

# 1) PROMPT ŞABLONLARI
# ---------------------

DRAFT_PROMPT = """
Aşağıdaki matematik problemini çöz.

Kurallar:
- Çözümü kısa ve net yaz.
- Gereksiz açıklama yazma.
- Adım adım hesap yapabilirsin ama çok uzatma.
- En sonda tek satırda şu formatta bitir:
FINAL: <sayı>

Soru:
{question}
""".strip()

JUDGE_PROMPT = """
Sen bir matematik denetmenisin.

Aşağıda bir matematik sorusu ve bu soruya verilen bir çözüm var.
Görevin, SADECE verilen çözümü değerlendirmektir.

Kurallar:
- Soruyu baştan çözmeye çalışma.
- Verilen çözümdeki işlemlerin ve ara sonuçların tutarlı olup olmadığına bak.
- Son satırdaki FINAL sonucunun, çözümdeki adımlarla uyumlu olup olmadığını kontrol et.
- Cevap makul ve tutarlı görünüyorsa "EVET" de.
- Bariz hesap hatası, mantık hatası veya eksik çözüm varsa "HAYIR" de.

Cevabın SADECE tek bir kelime olsun:
- EVET   (eğer çözüm doğru ve tutarlıysa)
- HAYIR  (eğer çözümde hata varsa)

Başka hiçbir şey yazma. Açıklama yazma. Sadece EVET veya HAYIR.

Soru:
{question}

Cevap:
{draft}
""".strip()

IMPROVE_PROMPT = """
Aşağıda bir matematik sorusu, bu soruya verilen önceki bir cevap
ve bu cevabın neden iyi olmadığına dair bir değerlendirme var.

Görevin:
- Soruyu baştan çöz.
- Hataları düzelt.
- Çözümü 3–6 kısa adımda yaz.
- Gereksiz açıklama yazma.
- En sonda tek satırda şu formatta bitir:
FINAL: <sayı>

Soru:
{question}

Önceki cevap:
{draft}

Değerlendirme:
{judge}
""".strip()

# 2) ORTAK YARDIMCI FONKSİYONLAR
# -------------------------------

def clean_model_output(text: str) -> str:
    if text is None:
        return ""
    if "</think>" in text:
        # Önceki [0] yerine [-1] → </think>’ten SONRAKİ kısmı al
        text = text.split("</think>")[-1]
    return text.strip()

def format_prompt(template, **kwargs):
    return template.format(**kwargs).strip() + "\n"

@torch.no_grad()
def generate_one(prompt, model, tokenizer,
                 max_new_tokens=256,
                 temperature=0.2,
                 top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        top_p=top_p,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    text = tokenizer.decode(out[0], skip_special_tokens=True)

    if text.startswith(prompt):
        text = text[len(prompt):]

    text = clean_model_output(text)
    return text.strip()

def extract_final_from_model(text):
    """
    Model çıktısından 'FINAL: ...' kısmını çekmeye çalışır.
    Önce FINAL satırlarını arar:
      - İçinde rakam olan FINAL satırları arasından SONUNCUSUNU seçer.
      - Eğer hiç rakamlı FINAL satırı yoksa:
          * Son FINAL satırının içeriğini dener
          * O da yoksa en son sayıyı alır.
    """
    if text is None:
        return None

    # Tüm FINAL: ... satırlarını bul
    matches = re.findall(r"FINAL:\s*(.+)", text, flags=re.IGNORECASE)

    if matches:
        numeric_candidates = []
        for m in matches:
            line = m.strip().splitlines()[0].strip()  # sadece ilk satır
            # Satırda herhangi bir rakam geçiyorsa numeric kabul et
            if re.search(r"\d", line):
                numeric_candidates.append(line)

        # İçinde rakam geçen FINAL satırları varsa -> sonuncusunu al
        if numeric_candidates:
            return numeric_candidates[-1]

        # Hiç rakamlı FINAL yok ama en az bir FINAL var -> sonuncu FINAL'ı al
        last = matches[-1].strip().splitlines()[0].strip()
        if last:
            return last

    # Buraya geldiysek: ya hiç FINAL yok, ya da işimize yaramadı.
    # Eski fallback: metindeki son sayıyı al.
    nums = re.findall(r"[-+]?\d+(?:[.,]\d+)?", text)
    if nums:
        return nums[-1].replace(",", ".").strip()

    return None

def extract_final_from_dataset(answer_field):
    """
    GSM8K-tr answer alanından doğru sayıyı çek.
    Genelde '#### 42' gibi olur, biz yine son sayıyı alıyoruz.
    """
    if answer_field is None:
        return None
    nums = re.findall(r"[-+]?\d+(?:[.,]\d+)?", answer_field)
    if nums:
        return nums[-1].replace(",", ".").strip()
    return None

def normalize_ans(ans):
    """
    10, 10.0, 10.00 gibi cevapları normalize eder ki karşılaştırma düzgün olsun.
    """
    if ans is None:
        return None
    a = str(ans).strip().replace(" ", "")
    # Sayı mı?
    if re.fullmatch(r"[-+]?\d+(\.\d+)?", a):
        try:
            x = float(a)
            if abs(x - round(x)) < 1e-9:
                return str(int(round(x)))
            return str(x)
        except:
            return a
    return a

def exact_match(pred, gold):
    p = normalize_ans(pred)
    g = normalize_ans(gold)
    return (p is not None) and (g is not None) and (p == g)

def parse_yes_no_from_judge(judge_text):
    """
    JUDGE çıktısından EVET / HAYIR etiketini çıkar.
    Önce açıkça EVET/HAYIR kelimesini arar,
    yoksa anlamına göre tahmin etmeye çalışır.
    """
    if judge_text is None:
        return None

    t = judge_text.strip().lower()

    # 1) Direkt EVET / HAYIR kelimesi var mı?
    words = t.split()
    first_word = words[0] if words else ""

    if first_word.startswith("evet"):
        return "EVET"
    if first_word.startswith("hayir") or first_word.startswith("hayır"):
        return "HAYIR"

    if "evet" in t and "hayir" not in t and "hayır" not in t:
        return "EVET"
    if "hayir" in t or "hayır" in t:
        return "HAYIR"

    # 2) Anlama göre heuristic:
    negative_keywords = ["hata", "yanlış", "yanlis", "tutarsız", "tutarsiz", "uyumsuz", "eksik"]
    positive_keywords = ["doğru", "dogru", "tutarlı", "tutarlı", "mantıklı", "mantikli"]

    if any(kw in t for kw in negative_keywords):
        return "HAYIR"
    if any(kw in t for kw in positive_keywords):
        return "EVET"

    return None

# 3) DRAFT / JUDGE / IMPROVE PIPELINE
# ------------------------------------

def answer_with_draft(question, model, tokenizer):
    prompt = format_prompt(DRAFT_PROMPT, question=question)
    draft = generate_one(prompt, model, tokenizer,
                         max_new_tokens=160,
                         temperature=0.2,
                         top_p=0.9)
    draft_final = extract_final_from_model(draft)
    return draft, draft_final

def judge_answer(question, draft, model, tokenizer):
    prompt = format_prompt(JUDGE_PROMPT, question=question, draft=draft)
    judge = generate_one(
        prompt,
        model,
        tokenizer,
        max_new_tokens=32,
        temperature=0.7,
        top_p=0.9
    )

    if judge is None or not judge.strip():
        return "", None

    label = parse_yes_no_from_judge(judge)
    return judge, label

def improve_answer(question, draft, judge, model, tokenizer):
    prompt = format_prompt(IMPROVE_PROMPT,
                           question=question,
                           draft=draft,
                           judge=judge)
    improved = generate_one(prompt, model, tokenizer,
                            max_new_tokens=160,
                            temperature=0.2,
                            top_p=0.9)
    improved_final = extract_final_from_model(improved)
    return improved, improved_final

def answer_with_self_improve(question, model, tokenizer):
    # 1) DRAFT
    draft, draft_final = answer_with_draft(question, model, tokenizer)

    # 2) JUDGE
    judge, label = judge_answer(question, draft, model, tokenizer)
    # label ∈ {"EVET", "HAYIR", None}

    # 2.5) JUDGE belirsizse:
    # Eğer draft_final VARSA → EVET (cevabı kabul et)
    # Eğer draft_final YOKSA → HAYIR (iyileştirmeye zorla)
    if label is None:
        if draft_final is not None:
            label = "EVET"
        else:
            label = "HAYIR"

    # 3) Gerekirse IMPROVE
    if label == "EVET":
        improved = None
        improved_final = None
        chosen_text = draft
        chosen_final = draft_final
    else:  # label == "HAYIR"
        improved, improved_final = improve_answer(question, draft, judge, model, tokenizer)
        chosen_text = improved if improved is not None else draft
        chosen_final = improved_final if improved_final is not None else draft_final

    return {
        "draft_text": draft,
        "draft_final": draft_final,
        "judge_text": judge,
        "judge_label": label,
        "improved_text": improved,
        "improved_final": improved_final,
        "chosen_text": chosen_text,
        "chosen_final": chosen_final,
    }

# 4) 500 SORULUK ITERASYON İÇİN SFT VERİSİ
# -----------------------------------------

def build_sft_rows_for_iter(ds_500, model, tokenizer,
                            limit=500,
                            iter_name="iter1",
                            debug_first_n=3):

    rows = []
    n = min(limit, len(ds_500))

    n_yes = 0
    n_no = 0
    n_unknown = 0
    n_used_draft = 0
    n_used_improved = 0
    n_skipped_no_final = 0   # istatistik için

    for i in tqdm(range(n), desc=f"Self-improve {iter_name}"):
        ex = ds_500[i]
        q = ex["question"]
        gold_final = extract_final_from_dataset(ex["answer"])

        res = answer_with_self_improve(q, model, tokenizer)

        # Orijinal judge label'ı istatistik için tut
        label = res["judge_label"]
        if label == "EVET":
            n_yes += 1
        elif label == "HAYIR":
            n_no += 1
        else:
            n_unknown += 1

        # --- ÖN TEMİZLİK: hiç sayısal FINAL yoksa bu örneği tamamen at ---
        if res["draft_final"] is None and res["improved_final"] is None:
            n_skipped_no_final += 1
            continue

        # DRAFT / IMPROVE doğruluk kontrolü (gold'a göre)
        is_draft_correct = exact_match(res["draft_final"], gold_final)
        is_improved_correct = exact_match(res["improved_final"], gold_final) if res["improved_final"] is not None else False

        # --- SFT için hangi cevabı SEÇECEĞİMİZİ gold'a göre belirle ---
        if is_draft_correct and not is_improved_correct:
            chosen_text = res["draft_text"]
            n_used_draft += 1
        elif is_improved_correct and not is_draft_correct:
            chosen_text = res["improved_text"]
            n_used_improved += 1
        elif is_draft_correct and is_improved_correct:
            chosen_text = res["draft_text"]
            n_used_draft += 1
        else:
            # İkisi de yanlışsa, pipeline'ın seçtiğini kullan
            chosen_text = res["chosen_text"]
            if label == "EVET":
                n_used_draft += 1
            else:
                n_used_improved += 1

        # --- KISA / BOŞ CEVAP TEMİZLİĞİ ---
        if chosen_text is None:
            continue
        chosen_clean = chosen_text.strip()
        if len(chosen_clean) < 20:   # çok kısa, muhtemelen saçma
            n_skipped_no_final += 1
            continue

        rows.append({
            "instruction": q,
            "response": chosen_clean
        })

        # Debug çıktıları
        if i < debug_first_n:
            print("\n====================")
            print(f"Soru {i+1}:")
            print("Soru:", q)
            print("Gold:", gold_final)
            print("\nDRAFT:\n", res["draft_text"])
            print("DRAFT FINAL:", res["draft_final"], "| Doğru mu?:", is_draft_correct)
            print("\nJUDGE:\n", res["judge_text"])
            print("LABEL:", res["judge_label"])
            if res["improved_text"] is not None:
                print("\nIMPROVED:\n", res["improved_text"])
                print("IMPROVED FINAL:", res["improved_final"], "| Doğru mu?:", is_improved_correct)
            print("\nSeçilen metin (eğitim için):\n", chosen_clean)
            print("====================")

    print("\n--- Iterasyon istatistikleri ---")
    print(f"Toplam soru          : {n}")
    print(f"Judge EVET sayısı    : {n_yes}")
    print(f"Judge HAYIR sayısı   : {n_no}")
    print(f"Judge UNKNOWN sayısı : {n_unknown}")
    print(f"Hiç FINAL üretemeyen/çok kısa olduğu için atılan soru sayısı : {n_skipped_no_final}")
    print()
    print(f"Eğitimde DRAFT kullanılan soru sayısı   : {n_used_draft}")
    print(f"Eğitimde IMPROVE kullanılan soru sayısı : {n_used_improved}")
    print(f"Toplam SFT satırı                        : {len(rows)}")

    return rows



In [24]:
# Tek soruluk test: modelin tüm süreci
idx = 57
ex = test_500[idx]
q = ex["question"]
gold = extract_final_from_dataset(ex["answer"])

print("=== TEK SORULUK SELF-IMPROVE TESTİ ===")
print(f"Soru index: {idx}")
print("Soru:")
print(q)
print("\nDoğru cevap (gold) [sadece analiz için]:", gold)

res = answer_with_self_improve(q, model, tokenizer)

print("\n--- DRAFT CEVAP ---")
print(res["draft_text"])
print("DRAFT FINAL:", res["draft_final"])

print("\n--- JUDGE CEVAP ---")
print(res["judge_text"])
print("JUDGE LABEL:", res["judge_label"])

print("\n--- IMPROVED CEVAP ---")
print(res["improved_text"])
print("IMPROVED FINAL:", res["improved_final"])

print("\n--- EĞİTİM İÇİN SEÇİLEN CEVAP ---")
print(res["chosen_text"])
print("CHOSEN FINAL:", res["chosen_final"])

draft_ok = exact_match(res["draft_final"], gold)
chosen_ok = exact_match(res["chosen_final"], gold)

print("\nDRAFT DOĞRU MU?      :", draft_ok)
print("CHOSEN (eğitim) DOĞRU MU? :", chosen_ok)


=== TEK SORULUK SELF-IMPROVE TESTİ ===
Soru index: 57
Soru:
Charlotte bir köpek gezdiricisi ve bu hafta tüm müşterileri için bir program oluşturuyor. Her köpek ayrı ayrı gezdiriliyor. Pazartesi günü 4 kaniş ve 2 Chihuahua'yı gezdirmesi gerekiyor. Salı günü, aynı miktarda Chihuahua'yı gezdiriyor ancak kaç tane kaniş gezdirmesi gerektiğinden emin değil. Çarşamba günü, 4 Labrador'u gezdiriyor. Bir kanişi gezdirmek 2 saat, bir Chihuahua'yı gezdirmek 1 saat ve bir Labrador'u gezdirmek 3 saat sürüyor. Bu hafta köpek gezdirmek için toplam 32 saat zamanı varsa, Charlotte Salı günü kaç kaniş gezdirebilir?

Doğru cevap (gold) [sadece analiz için]: 4

--- DRAFT CEVAP ---
Soru Salı günü kaç kaniş gezdirebileceğini soruyor.

Çözüm:
Öncelikle, Pazartesi günü: 4 kaniş ve 2 Chihuahua.
Kaniş başına 2 saat, Chihuahua başına 1 saat.
Pazartesi toplam süre: (4 * 2) + (2 * 1) = 8 + 2 = 10 saat.

Çarşamba günü: 4 Labrador.
Labrador başına 3 saat, yani 4 * 3 = 12 saat.

Şimdi, Salı günü aynı miktarda Chihuahu

In [19]:
from tqdm import tqdm

def evaluate_on_test(test_ds, model, tokenizer, mode="draft", limit=None):
    """
    test_ds     : test_500
    mode        : "draft" veya "self_improve"
    limit       : None -> tüm test_ds,
                  sayı -> ilk 'limit' soru
    """
    n = len(test_ds) if limit is None else min(limit, len(test_ds))
    correct = 0
    total = 0

    for i in tqdm(range(n), desc=f"Eval ({mode}, n={n})"):
        ex = test_ds[i]
        q = ex["question"]
        gold = extract_final_from_dataset(ex["answer"])

        if mode == "draft":
            draft_text, draft_final = answer_with_draft(q, model, tokenizer)
            pred = draft_final
        elif mode == "self_improve":
            res = answer_with_self_improve(q, model, tokenizer)
            pred = res["chosen_final"]
        else:
            raise ValueError("mode 'draft' veya 'self_improve' olmalı")

        if exact_match(pred, gold):
            correct += 1
        total += 1

    acc = correct / total if total > 0 else 0.0
    return {"mode": mode, "n": total, "acc": acc, "correct": correct}


In [20]:
# 500 soruluk sabit test seti üzerinde BAŞLANGIÇ performansı ölçelim

# 1) Sadece draft modu (baseline)
result_test_draft_500 = evaluate_on_test(
    test_ds=test_500,
    model=model,
    tokenizer=tokenizer,
    mode="draft",
    limit=100
)

# 2) Self-improve pipeline (draft + judge + improve + chosen_final)
result_test_self_500 = evaluate_on_test(
    test_ds=test_500,
    model=model,
    tokenizer=tokenizer,
    mode="self_improve",
    limit=50
)

print("\n--- Özet ---")
print("Draft mode  : ", result_test_draft_500)
print("Self-improve: ", result_test_self_500)


Eval (self_improve, n=50): 100%|██████████| 50/50 [12:29<00:00, 14.99s/it]


--- Özet ---
Draft mode  :  {'mode': 'draft', 'n': 100, 'acc': 0.49, 'correct': 49}
Self-improve:  {'mode': 'self_improve', 'n': 50, 'acc': 0.46, 'correct': 23}


In [21]:
sft_iter1 = build_sft_rows_for_iter(
    ds_500=iter1,
    model=model,
    tokenizer=tokenizer,
    limit=500,
    iter_name="iter1",
    debug_first_n=3
)

len(sft_iter1)


Self-improve iter1:   0%|          | 1/500 [00:18<2:36:35, 18.83s/it]


Soru 1:
Soru: Nancy'nin eski kocasının ona nafaka borcu var. Her yıl gelirinin %30'unu ödemesi gerekiyor. Üç yıl boyunca yılda 30.000 dolar kazanmış, sonraki dört yıl boyunca %20 zam almış. Şimdiye kadar sadece 1200 dolar ödediyse, ona ne kadar borcu var?
Gold: 69.000

DRAFT:
 
DRAFT FINAL: None | Doğru mu?: False

JUDGE:
 Çözüm:
Nancy'nin üç yıl boyunca yılda 30,000 dolar kazandığını biliyoruz.
Sonraki dört
LABEL: HAYIR

IMPROVED:
 yıl boyunca %20 zam almış.
Şimdiye kadar sadece 1200 dolar ödediyse, ona ne kadar borcu var?
Önceki cevap:
Nancy'nin üç yıl boyunca yılda 30,000 dolar kazandığını biliyoruz.
Sonraki dört
yıl boyunca %20 zam almış.
Şimdiye kadar sadece 1200 dolar ödediyse, ona ne kadar borcu var?
Önceki cevap:
Nancy'nin üç yıl boyunca yılda 30,000 dolar kazandığını biliyoruz.
Sonraki dört
yıl boyunca %20 zam almış.
Şimdiye kadar sadece 1200 dolar ödediyse, ona ne kadar borcu var?
Önceki cevap:
Nancy'nin üç yıl boyunca yılda 30,000 dolar kazandığını biliyoruz.
Sonraki dört
y

Self-improve iter1:   0%|          | 2/500 [00:37<2:35:01, 18.68s/it]


Soru 2:
Soru: Bir araba her 2 milde bir 725 kez lastik değiştirir.  Jeremy ayda 400 mil araba kullanıyor.  Her bir lastik 10.440.000 kez dönebiliyorsa, lastiğin değiştirilmesi gerekmeden önce kaç yıl geçmesi gerekir?
Gold: 6

DRAFT:
 Jeremy ayda 400 mil araba kullanıyor, yani yılda 400 * 12 = 4800 mil kullanıyor.  
Araba her 2 milde bir lastik değiştiriyor, yani lastik ömrü 2 mil.  
Bu durumda bir lastiğin değiştirilmesi gereken mesafe 2 mil.  
Jeremy yılda 4800 mil kullanıyor, yani bir lastiğin ömrü 4800 / 2 = 2400 yılda bir değiştirilmesi gerekir.  
Ancak soruda lastiklerin 10.440.000 kez dönebildiği verilmiş.  
Lastik dönüş sayısı ile mesafe arasında ilişki kurmamız gerekiyor.  
Bir lastiğin çevresini bilmiyoruz, ancak dönüş sayısı verildiğine göre, mesafeyi dönüş sayısı cinsinden ifade edebiliriz.  
Diyelim ki bir lastiğin çevresi C mil olsun. O zaman bir dönüşte alınan mesafe C mil olur.  
Lastik değiştirilmesi gereken mesafe 2 mil, yani dönüş sayısı olarak 2 / C olur.
DRAFT FINA

Self-improve iter1:   1%|          | 3/500 [00:44<1:49:42, 13.25s/it]


Soru 3:
Soru: Reggie'nin babası ona 48 dolar verdi. Reggie her biri 2 dolar olan 5 kitap aldı. Reggie'nin ne kadar parası kaldı?
Gold: 38

DRAFT:
 Reggie'nin başlangıçta 48 doları var.  
5 kitap alıyor, her kitap 2 dolar, toplam 5 × 2 = 10 dolar ödüyor.  
Kalan para: 48 - 10 = 38 dolar.  

FINAL: 38
DRAFT FINAL: 38 | Doğru mu?: True

JUDGE:
 ---

Şimdi değerlendirme yap:

- Çözüm, soruda verilenleri kullanıyor: 48 dolar başlangıç,
LABEL: EVET

Seçilen metin (eğitim için):
 Reggie'nin başlangıçta 48 doları var.  
5 kitap alıyor, her kitap 2 dolar, toplam 5 × 2 = 10 dolar ödüyor.  
Kalan para: 48 - 10 = 38 dolar.  

FINAL: 38


Self-improve iter1:  57%|█████▋    | 284/500 [1:15:41<57:33, 15.99s/it]


KeyboardInterrupt: 

In [42]:
import re
from tqdm import tqdm
import torch

# 1) PROMPT ŞABLONLARI


DRAFT_PROMPT = """
Aşağıdaki matematik problemini çöz.

Kurallar:
- Çözümü kısa ve net yaz.
- Gereksiz açıklama yazma.
- Adım adım hesap yapabilirsin ama çok uzatma.
- En sonda tek satırda şu formatta bitir:
FINAL: <sayı>

Soru:
{question}
""".strip()

JUDGE_PROMPT = """
Sen bir matematik denetmenisin.

Aşağıda bir matematik sorusu ve bu soruya verilen bir çözüm var.
Görevin, SADECE verilen çözümü değerlendirmektir.

Kurallar:
- Soruyu baştan çözme.
- Verilen çözümdeki işlemlerin ve FINAL sonucunun tutarlı olup olmadığına bak.
- Her şey mantıklı ve FINAL doğru görünüyorsa "EVET" de.
- Bariz hesap hatası, mantık hatası veya FINAL yanlışsa "HAYIR" de.

Cevabın yalnızca bir kelime olsun:
EVET
veya
HAYIR

Başka hiçbir şey yazma.

Soru:
{question}

Çözüm:
{draft}
""".strip()

IMPROVE_PROMPT = """
Aşağıda bir matematik sorusu, bu soruya verilen önceki bir cevap
ve bu cevabın neden iyi olmadığına dair bir değerlendirme var.

Görevin:
- Soruyu baştan çöz.
- Hataları düzelt.
- Çözümü 3–6 kısa adımda yaz.
- Gereksiz açıklama yazma.
- En sonda tek satırda şu formatta bitir:
FINAL: <sayı>

Soru:
{question}

Önceki cevap:
{draft}

Değerlendirme:
{judge}
""".strip()

# 2) ORTAK YARDIMCI FONKSİYONLAR


def clean_model_output(text: str) -> str:
    """</think> varsa, ondan SONRASINI al (gizli düşünceyi at)."""
    if text is None:
        return ""
    if "</think>" in text:
        text = text.split("</think>")[-1]
    return text.strip()

def format_prompt(template, **kwargs):
    return template.format(**kwargs).strip() + "\n"

@torch.no_grad()
def generate_one(prompt, model, tokenizer,
                 max_new_tokens=256,
                 temperature=0.2,
                 top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0.0),
        temperature=temperature,
        top_p=top_p,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    if text.startswith(prompt):
        text = text[len(prompt):]
    text = clean_model_output(text)
    return text.strip()

def extract_final_from_model(text):
    """
    Model çıktısından 'FINAL: ...' kısmını çekmeye çalışır.
    Önce FINAL satırlarını arar:
      - İçinde rakam olan FINAL satırları arasından SONUNCUSUNU seçer.
      - Eğer hiç rakamlı FINAL satırı yoksa:
          * Son FINAL satırının içeriğini dener
          * O da yoksa metindeki son sayıyı alır.
    """
    if text is None:
        return None

    matches = re.findall(r"FINAL:\s*(.+)", text, flags=re.IGNORECASE)
    if matches:
        numeric_candidates = []
        for m in matches:
            line = m.strip().splitlines()[0].strip()
            if re.search(r"\d", line):
                numeric_candidates.append(line)
        if numeric_candidates:
            return numeric_candidates[-1]
        last = matches[-1].strip().splitlines()[0].strip()
        if last:
            return last

    nums = re.findall(r"[-+]?\d+(?:[.,]\d+)?", text)
    if nums:
        return nums[-1].replace(",", ".").strip()
    return None

def extract_final_from_dataset(answer_field):
    """GSM8K-tr 'answer' alanından doğru sayıyı çek (son sayıyı al)."""
    if answer_field is None:
        return None
    nums = re.findall(r"[-+]?\d+(?:[.,]\d+)?", answer_field)
    if nums:
        return nums[-1].replace(",", ".").strip()
    return None

def normalize_ans(ans):
    """10, 10.0, 10.00 gibi cevapları normalize eder."""
    if ans is None:
        return None
    a = str(ans).strip().replace(" ", "")
    if re.fullmatch(r"[-+]?\d+(\.\d+)?", a):
        try:
            x = float(a)
            if abs(x - round(x)) < 1e-9:
                return str(int(round(x)))
            return str(x)
        except:
            return a
    return a

def exact_match(pred, gold):
    p = normalize_ans(pred)
    g = normalize_ans(gold)
    return (p is not None) and (g is not None) and (p == g)

def parse_yes_no_from_judge(judge_text):
    """
    JUDGE çıktısından EVET / HAYIR etiketini çıkar.
    Önce ilk kelimeye bak, sonra metnin tamamına bak.
    """
    if judge_text is None:
        return None
    t = judge_text.strip().lower()
    words = t.split()
    first_word = words[0] if words else ""

    if first_word.startswith("evet"):
        return "EVET"
    if first_word.startswith("hayir") or first_word.startswith("hayır"):
        return "HAYIR"

    if "evet" in t and "hayir" not in t and "hayır" not in t:
        return "EVET"
    if "hayir" in t or "hayır" in t:
        return "HAYIR"

    return None


# 3) DRAFT / JUDGE / IMPROVE PIPELINE

def answer_with_draft(question, model, tokenizer):
    prompt = format_prompt(DRAFT_PROMPT, question=question)
    draft = generate_one(prompt, model, tokenizer,
                         max_new_tokens=256,
                         temperature=0.2,
                         top_p=0.9)
    draft_final = extract_final_from_model(draft)
    return draft, draft_final

def judge_answer(question, draft, model, tokenizer):
    """
    Modelden sadece EVET / HAYIR bekliyoruz.
    """
    prompt = format_prompt(JUDGE_PROMPT, question=question, draft=draft)
    judge = generate_one(
        prompt,
        model,
        tokenizer,
        max_new_tokens=8,
        temperature=0.0,  # deterministik
        top_p=1.0
    )
    if judge is None or not judge.strip():
        return "", None
    label = parse_yes_no_from_judge(judge)
    return judge, label

def improve_answer(question, draft, judge, model, tokenizer):
    prompt = format_prompt(IMPROVE_PROMPT,
                           question=question,
                           draft=draft,
                           judge=judge)
    improved = generate_one(prompt, model, tokenizer,
                            max_new_tokens=256,
                            temperature=0.2,
                            top_p=0.9)
    improved_final = extract_final_from_model(improved)
    return improved, improved_final

def answer_with_self_improve(question, model, tokenizer, gold_final=None):
    """
    Self-improve süreci:
      1) DRAFT cevabı üret
      2) Eğer gold_final verilmiş VE draft doğruysa:
           -> Judge/improve yapmadan direkt EVET + draft dön (hız için)
      3) Aksi halde:
           - Judge (EVET/HAYIR)
           - HAYIR ise improve çağır
    """
    # 1) DRAFT
    draft_text, draft_final = answer_with_draft(question, model, tokenizer)

    # 1.5) Gold varsa ve draft zaten doğruysa → direkt EVET
    if gold_final is not None and exact_match(draft_final, gold_final):
        return {
            "draft_text": draft_text,
            "draft_final": draft_final,
            "judge_text": "EVET",
            "judge_label": "EVET",
            "improved_text": None,
            "improved_final": None,
            "chosen_text": draft_text,
            "chosen_final": draft_final,
        }

    # 2) JUDGE
    judge_text, label = judge_answer(question, draft_text, model, tokenizer)

    # 2.5) Judge belirsizse → HAYIR varsay
    if label is None:
        label = "HAYIR"

    # 3) Gerekirse IMPROVE
    if label == "EVET":
        improved_text = None
        improved_final = None
        chosen_text = draft_text
        chosen_final = draft_final
    else:
        improved_text, improved_final = improve_answer(
            question, draft_text, judge_text, model, tokenizer
        )
        chosen_text = improved_text if improved_text is not None else draft_text
        chosen_final = improved_final if improved_final is not None else draft_final

    return {
        "draft_text": draft_text,
        "draft_final": draft_final,
        "judge_text": judge_text,
        "judge_label": label,
        "improved_text": improved_text,
        "improved_final": improved_final,
        "chosen_text": chosen_text,
        "chosen_final": chosen_final,
    }

def build_sft_rows_for_iter(
    ds_500,
    model,
    tokenizer,
    limit=500,
    iter_name="iter1",
    debug_first_n=3
):
    """
    500 soruluk bir iterasyon için SFT eğitim verisini hazırlar.

    - Her soru için draft + judge + (gerekirse) improve çalıştırılır.
    - Gold cevap (dataset'ten) sadece HANGİ cevabı seçeceğine karar vermek için kullanılır.
    - Bozuk / anlamsız / FINAL'siz cevaplar filtrelenir.
    - Çıktı: [{"instruction": soru, "response": seçilen_cevap}, ...]
    """
    rows = []
    n = min(limit, len(ds_500))

    n_yes = 0
    n_no = 0
    n_unknown = 0
    n_used_draft = 0
    n_used_improved = 0
    n_dropped = 0

    for i in tqdm(range(n), desc=f"Self-improve {iter_name}"):
        ex = ds_500[i]
        q = ex["question"]
        gold_final = extract_final_from_dataset(ex["answer"])

        # Tüm pipeline (burada gold_final'i veriyoruz -> hızlı shortcut)
        res = answer_with_self_improve(q, model, tokenizer, gold_final=gold_final)

        # İstatistik için judge label
        label = res["judge_label"]
        if label == "EVET":
            n_yes += 1
        elif label == "HAYIR":
            n_no += 1
        else:
            n_unknown += 1

        # DRAFT / IMPROVE doğruluk (gold'a göre)
        is_draft_correct = exact_match(res["draft_final"], gold_final)
        is_improved_correct = exact_match(res["improved_final"], gold_final) if res["improved_final"] is not None else False

        # 1) Gold'a göre en iyi cevabı seç
        if is_draft_correct and not is_improved_correct:
            chosen_for_sft = res["draft_text"]
            n_used_draft += 1
        elif is_improved_correct and not is_draft_correct:
            chosen_for_sft = res["improved_text"]
            n_used_improved += 1
        elif is_draft_correct and is_improved_correct:
            chosen_for_sft = res["draft_text"]
            n_used_draft += 1
        else:
            # İkisi de yanlışsa, pipeline'ın seçtiğini kullan (draft/improve)
            chosen_for_sft = res["chosen_text"]
            if label == "EVET":
                n_used_draft += 1
            else:
                n_used_improved += 1

        # 2) TEMİZLİK: çöp verileri at
        if chosen_for_sft is None or not chosen_for_sft.strip():
            n_dropped += 1
            continue

        # Çok kısa/gereksiz metinler → at
        if len(chosen_for_sft.strip()) < 20:
            n_dropped += 1
            continue

        # İçinde FINAL geçen ve en azından bir sayı barındıran cevap olsun
        if "FINAL" not in chosen_for_sft:
            n_dropped += 1
            continue

        # 3) SFT satırına ekle
        rows.append({
            "instruction": q,
            "response": chosen_for_sft.strip()
        })

        # Debug için ilk birkaç soruyu yazdır
        if i < debug_first_n:
            print("\n====================")
            print(f"Soru {i+1}:")
            print("Soru:", q)
            print("Gold:", gold_final)
            print("\nDRAFT:\n", res["draft_text"])
            print("DRAFT FINAL:", res["draft_final"], "| Doğru mu?:", is_draft_correct)
            print("\nJUDGE:\n", res["judge_text"])
            print("LABEL:", res["judge_label"])
            if res["improved_text"] is not None:
                print("\nIMPROVED:\n", res["improved_text"])
                print("IMPROVED FINAL:", res["improved_final"], "| Doğru mu?:", is_improved_correct)
            print("\nSeçilen metin (SFT için):\n", chosen_for_sft)
            print("====================")

    print("\n--- Iterasyon istatistikleri ---")
    print(f"Toplam soru                     : {n}")
    print(f"Judge EVET sayısı               : {n_yes}")
    print(f"Judge HAYIR sayısı              : {n_no}")
    print(f"Judge UNKNOWN sayısı            : {n_unknown}")
    print()
    print(f"SFT'de DRAFT kullanılan soru sayısı   : {n_used_draft}")
    print(f"SFT'de IMPROVE kullanılan soru sayısı : {n_used_improved}")
    print(f"Filtrelenip atılan soru sayısı        : {n_dropped}")
    print(f"Toplam SFT satırı (rows)              : {len(rows)}")

    return rows


In [43]:
idx = 0
ex = test_500[idx]
q = ex["question"]
gold = extract_final_from_dataset(ex["answer"])

print("Soru:", q)
print("Gold:", gold)

res = answer_with_self_improve(q, model, tokenizer)
print("\nDRAFT:", res["draft_text"])
print("DRAFT FINAL:", res["draft_final"])
print("\nJUDGE:", res["judge_text"], "| LABEL:", res["judge_label"])
print("\nIMPROVED:", res["improved_text"])
print("IMPROVED FINAL:", res["improved_final"])
print("\nCHOSEN FINAL:", res["chosen_final"])
print("DOĞRU MU?:", exact_match(res["chosen_final"], gold))


Soru: Bir tersanedeki tank sayısı kamyon sayısının beş katıdır. Avluda 20 kamyon varsa, avludaki toplam tank ve kamyon sayısını hesaplayın.
Gold: 120

DRAFT: Tank sayısı kamyon sayısının 5 katıdır.  
Avluda 20 kamyon olduğuna göre, tank sayısı = 20 × 5 = 100.  
Toplam araç sayısı = tank sayısı + kamyon sayısı = 100 + 20 = 120.  

FINAL: 120
DRAFT FINAL: 120

JUDGE: ---

Şimdi değerlendirme yap | LABEL: HAYIR

IMPROVED: Önceki cevap doğru görünüyor ama soruda "avludaki toplam tank ve kamyon sayısı" ifadesi var. Önceki cevap sadece tank ve kamyonları toplamış, fakat soruda "tank ve kamyon" olarak belirtilmiş, yani sadece bu ikisini topluyor. Önceki cevapta bir hata yok.

Ama değerlendirme kısmında "avludaki toplam tank ve kamyon sayısı" ifadesi var ve önceki cevap bunu yapmış. Değerlendirme kısmında "Önceki cevap doğru görünüyor" denmiş. Fakat değerlendirme kısmında "fakat" ile başlayan bir şey yok. Değerlendirme kısmı sadece "Önceki cevap doğru görünüyor" diyor.

Değerlendirme kısmı:
"Ö

In [44]:
sft_rows_iter1 = build_sft_rows_for_iter(
    ds_500=iter1,
    model=model,
    tokenizer=tokenizer,
    limit=500,
    iter_name="iter1",
    debug_first_n=3
)

len(sft_rows_iter1)


Self-improve iter1:   1%|          | 3/500 [01:16<2:57:58, 21.49s/it]


Soru 3:
Soru: Reggie'nin babası ona 48 dolar verdi. Reggie her biri 2 dolar olan 5 kitap aldı. Reggie'nin ne kadar parası kaldı?
Gold: 38

DRAFT:
 Reggie'nin başlangıçta 48 doları var.  
5 kitap alıyor, her kitap 2 dolar, yani 5 × 2 = 10 dolar ödüyor.  
Kalan para: 48 - 10 = 38 dolar.  

FINAL: 38
DRAFT FINAL: 38 | Doğru mu?: True

JUDGE:
 EVET
LABEL: EVET

Seçilen metin (SFT için):
 Reggie'nin başlangıçta 48 doları var.  
5 kitap alıyor, her kitap 2 dolar, yani 5 × 2 = 10 dolar ödüyor.  
Kalan para: 48 - 10 = 38 dolar.  

FINAL: 38


Self-improve iter1: 100%|██████████| 500/500 [2:58:57<00:00, 21.48s/it]


--- Iterasyon istatistikleri ---
Toplam soru                     : 500
Judge EVET sayısı               : 242
Judge HAYIR sayısı              : 258
Judge UNKNOWN sayısı            : 0

SFT'de DRAFT kullanılan soru sayısı   : 242
SFT'de IMPROVE kullanılan soru sayısı : 258
Filtrelenip atılan soru sayısı        : 256
Toplam SFT satırı (rows)              : 244


244

In [58]:
import json
import os

# SFT veri klasörü
sft_dir = "/content/drive/MyDrive/kolektif_ogrenme_sft"
os.makedirs(sft_dir, exist_ok=True)

# 1. iterasyon SFT satırlarını JSONL olarak kaydet
iter1_path = os.path.join(sft_dir, "sft_iter1.jsonl")

with open(iter1_path, "w", encoding="utf-8") as f:
    for row in sft_rows_iter1:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

print("Iter1 SFT dosyası kaydedildi:", iter1_path)
print("Iter1 SFT satır sayısı:", len(sft_rows_iter1))

Iter1 SFT dosyası kaydedildi: /content/drive/MyDrive/kolektif_ogrenme_sft/sft_iter1.jsonl
Iter1 SFT satır sayısı: 244


In [48]:
RUN_ITER2 = False
RUN_ITER3 = False

sft_rows_iter2, sft_rows_iter3 = [], []

if RUN_ITER2:
    sft_rows_iter2 = build_sft_rows_for_iter(
        ds_500=iter2,
        model=model,
        tokenizer=tokenizer,
        limit=500,
        iter_name="iter2",
        debug_first_n=1
    )
    iter2_path = os.path.join(sft_dir, "sft_iter2.jsonl")
    with open(iter2_path, "w", encoding="utf-8") as f:
        for row in sft_rows_iter2:
            f.write(json.dumps(row, ensure_ascii=False) + "\n")
    print("Iter2 SFT dosyası kaydedildi:", iter2_path)
    print("Iter2 SFT satır sayısı:", len(sft_rows_iter2))

if RUN_ITER3:
    sft_rows_iter3 = build_sft_rows_for_iter(
        ds_500=iter3,
        model=model,
        tokenizer=tokenizer,
        limit=500,
        iter_name="iter3",
        debug_first_n=1
    )
    iter3_path = os.path.join(sft_dir, "sft_iter3.jsonl")
    with open(iter3_path, "w", encoding="utf-8") as f:
        for row in sft_rows_iter3:
            f.write(json.dumps(row, ensure_ascii=False) + "\n")
    print("Iter3 SFT dosyası kaydedildi:", iter3_path)
    print("Iter3 SFT satır sayısı:", len(sft_rows_iter3))


In [49]:

all_sft_rows = []
all_sft_rows.extend(sft_rows_iter1)

if len(sft_rows_iter2) > 0:
    all_sft_rows.extend(sft_rows_iter2)
if len(sft_rows_iter3) > 0:
    all_sft_rows.extend(sft_rows_iter3)

print("Toplam SFT satırı (tüm iterasyonlar):", len(all_sft_rows))


Toplam SFT satırı (tüm iterasyonlar): 244


In [50]:
from datasets import Dataset

if len(all_sft_rows) == 0:
    print("UYARI: all_sft_rows boş, iterasyonları tam koşturamadım.")
else:
    sft_dataset = Dataset.from_list(all_sft_rows)
    print(sft_dataset)
    print("Örnek bir satır:")
    print(sft_dataset[0])


Dataset({
    features: ['instruction', 'response'],
    num_rows: 244
})
Örnek bir satır:
{'instruction': "Reggie'nin babası ona 48 dolar verdi. Reggie her biri 2 dolar olan 5 kitap aldı. Reggie'nin ne kadar parası kaldı?", 'response': "Reggie'nin başlangıçta 48 doları var.  \n5 kitap alıyor, her kitap 2 dolar, yani 5 × 2 = 10 dolar ödüyor.  \nKalan para: 48 - 10 = 38 dolar.  \n\nFINAL: 38"}


In [51]:
!pip install -q trl

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer
import math


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 20.5 MB/s eta 0:00:00


In [52]:
# LoRA konfigürasyonu (örnek değerler)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


In [53]:
def format_sft_example(example):
    # Basit format: soru + iki satır boşluk + çözüm
    return f"Soru:\n{example['instruction']}\n\nCevap:\n{example['response']}"


In [54]:
if len(all_sft_rows) > 0:
    sft_dataset = sft_dataset.map(
        lambda ex: {"text": format_sft_example(ex)},
        remove_columns=["instruction", "response"]
    )
    print(sft_dataset[0]["text"][:500])


Map:   0%|          | 0/244 [00:00<?, ? examples/s]

Soru:
Reggie'nin babası ona 48 dolar verdi. Reggie her biri 2 dolar olan 5 kitap aldı. Reggie'nin ne kadar parası kaldı?

Cevap:
Reggie'nin başlangıçta 48 doları var.  
5 kitap alıyor, her kitap 2 dolar, yani 5 × 2 = 10 dolar ödüyor.  
Kalan para: 48 - 10 = 38 dolar.  

FINAL: 38


In [63]:
from datasets import Dataset  # eksikse lazım olacak

if len(all_sft_rows) == 0:
    print("SFT verisi yok, eğitim kodu sadece referans amaçlı.")
else:
    # 1) all_sft_rows → HF Dataset
    # all_sft_rows: [{"instruction": ..., "response": ...}, ...] formatında olmalı
    sft_dataset = Dataset.from_list(all_sft_rows)

    # 2) Her satıra 'text' alanı ekle:
    #    text = DRAFT_PROMPT(soruya göre) + response
    def add_text(example):
        prompt = DRAFT_PROMPT.format(question=example["instruction"]).strip() + "\n"
        example["text"] = prompt + example["response"].strip() + "\n"
        return example

    sft_dataset = sft_dataset.map(add_text)
    print("SFT dataset satır sayısı:", len(sft_dataset))

    # 3) Modeli LoRA için hazırlama (senin yazdığın şekliyle)
    base_model = model
    base_model = prepare_model_for_kbit_training(base_model)
    lora_model = get_peft_model(base_model, lora_config)

    output_dir = "/content/drive/MyDrive/kolektif_ogrenme_lora_iter1"

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        num_train_epochs=1,              # örnek olarak 1 epoch
        logging_steps=10,
        save_strategy="no",
        # bf16 GPU’ya göre sorun çıkarıyorsa False yapabilirsin:
        bf16=False,
        report_to="none"
    )

    trainer = SFTTrainer(
    model=lora_model,
    train_dataset=sft_dataset,
    processing_class=tokenizer,
    args=training_args
)




    print("LoRA SFT eğitimini başlatıyoruz...")
    trainer.train()

    # 4) Eğitilmiş modeli kaydet
    trainer.model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print("Eğitilmiş LoRA model kaydedildi:", output_dir)

    # 5) Sonraki iterasyon/testlerde bunu kullanmak istersen:
    model = trainer.model


Map:   0%|          | 0/244 [00:00<?, ? examples/s]

SFT dataset satır sayısı: 244


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/244 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/244 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/244 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 107}.


LoRA SFT eğitimini başlatıyoruz...


Step,Training Loss
10,1.115800
20,0.662700
30,0.646000
40,0.582900
50,0.542000
60,0.518700


Eğitilmiş LoRA model kaydedildi: /content/drive/MyDrive/kolektif_ogrenme_lora_iter1


In [64]:
from tqdm import tqdm

def evaluate_model_on_dataset(
    model,
    tokenizer,
    ds_500,
    use_self_improve=True,
    max_samples=None,
    desc="Eval"
):
    n = len(ds_500)
    if max_samples is not None:
        n = min(n, max_samples)

    correct = 0

    for i in tqdm(range(n), desc=desc):
        ex = ds_500[i]
        q = ex["question"]
        gold = extract_final_from_dataset(ex["answer"])

        if use_self_improve:
            res = answer_with_self_improve(q, model, tokenizer, gold_final=None)
            pred_final = res["chosen_final"]
        else:
            draft_text, draft_final = answer_with_draft(q, model, tokenizer)
            pred_final = draft_final

        if exact_match(pred_final, gold):
            correct += 1

    acc = correct / n if n > 0 else 0.0
    print(f"\n>>> {desc} | Doğru sayısı: {correct}/{n}  | Accuracy: {acc:.4f}")
    return {
        "accuracy": acc,
        "n": n,
        "correct": correct,
    }


In [67]:
# Self-improve pipeline ile iter1 sonrası test
iter1_eval_self = evaluate_model_on_dataset(
    model,
    tokenizer,
    test_500,
    use_self_improve=True,
    max_samples=50,
    desc="Iter1 - Self-improve"
)


iter1_eval_draft = evaluate_model_on_dataset(
    model,
    tokenizer,
    test_500,
    use_self_improve=False,
    max_samples=50,
    desc="Iter1 - Draft only"
)

print("Iter1 Self-improve:", iter1_eval_self)
print("Iter1 Draft only  :", iter1_eval_draft)


Iter1 - Self-improve: 100%|██████████| 5/5 [11:09<00:00, 133.97s/it]



>>> Iter1 - Self-improve | Doğru sayısı: 3/5  | Accuracy: 0.6000


Iter1 - Draft only: 100%|██████████| 5/5 [02:47<00:00, 33.50s/it]


>>> Iter1 - Draft only | Doğru sayısı: 5/5  | Accuracy: 1.0000
Iter1 Self-improve: {'accuracy': 0.6, 'n': 5, 'correct': 3}
Iter1 Draft only  : {'accuracy': 1.0, 'n': 5, 'correct': 5}


In [ ]:

sft_rows_iter2 = build_sft_rows_for_iter(
    ds_500=iter2,
    model=model,
    tokenizer=tokenizer,
    limit=500,
    iter_name="iter2",
    debug_first_n=3
)

print("Iter2 için üretilen SFT satırı sayısı:", len(sft_rows_iter2))

iter2_path = os.path.join(sft_dir, "sft_iter2.jsonl")
with open(iter2_path, "w", encoding="utf-8") as f:
    for row in sft_rows_iter2:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

print("Iter2 SFT dosyası kaydedildi:", iter2_path)


In [ ]:


from datasets import Dataset

sft_dataset_iter2 = Dataset.from_list(sft_rows_iter2)

def add_text_iter2(example):
    prompt = DRAFT_PROMPT.format(question=example["instruction"]).strip() + "\n"
    example["text"] = prompt + example["response"].strip() + "\n"
    return example

sft_dataset_iter2 = sft_dataset_iter2.map(add_text_iter2)
print("Iter2 SFT dataset satır sayısı:", len(sft_dataset_iter2))
print(sft_dataset_iter2[0]["text"][:300])


In [ ]:

output_dir_iter2 = "/content/drive/MyDrive/kolektif_ogrenme_lora_iter2"

training_args_iter2 = TrainingArguments(
    output_dir=output_dir_iter2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="no",
    bf16=False,
    report_to="none"
)

trainer_iter2 = SFTTrainer(
    model=model,
    train_dataset=sft_dataset_iter2,
    processing_class=tokenizer,
    args=training_args_iter2
)

print("Iter2 LoRA SFT eğitimini başlatıyoruz...")
trainer_iter2.train()

trainer_iter2.model.save_pretrained(output_dir_iter2)
tokenizer.save_pretrained(output_dir_iter2)
print("Eğitilmiş LoRA iter2 model kaydedildi:", output_dir_iter2)

model = trainer_iter2.model


In [ ]:

print("=== ITERATION 2 TEST SONUÇLARI ===")

iter2_eval_self = evaluate_model_on_dataset(
    model,
    tokenizer,
    test_500,
    use_self_improve=True,
    max_samples=50,
    desc="Iter2 - Self-improve"
)

iter2_eval_draft = evaluate_model_on_dataset(
    model,
    tokenizer,
    test_500,
    use_self_improve=False,
    max_samples=50,
    desc="Iter2 - Draft only"
)

print("\nIter2 Self-improve:", iter2_eval_self)
print("Iter2 Draft only  :", iter2_eval_draft)


In [ ]:

sft_rows_iter3 = build_sft_rows_for_iter(
    ds_500=iter3,
    model=model,
    tokenizer=tokenizer,
    limit=500,
    iter_name="iter3",
    debug_first_n=3
)

print("Iter3 için üretilen SFT satırı sayısı:", len(sft_rows_iter3))

iter3_path = os.path.join(sft_dir, "sft_iter3.jsonl")
with open(iter3_path, "w", encoding="utf-8") as f:
    for row in sft_rows_iter3:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

print("Iter3 SFT dosyası kaydedildi:", iter3_path)


In [ ]:

sft_dataset_iter3 = Dataset.from_list(sft_rows_iter3)

def add_text_iter3(example):
    prompt = DRAFT_PROMPT.format(question=example["instruction"]).strip() + "\n"
    example["text"] = prompt + example["response"].strip() + "\n"
    return example

sft_dataset_iter3 = sft_dataset_iter3.map(add_text_iter3)
print("Iter3 SFT dataset satır sayısı:", len(sft_dataset_iter3))
print(sft_dataset_iter3[0]["text"][:300])


In [ ]:

output_dir_iter3 = "/content/drive/MyDrive/kolektif_ogrenme_lora_iter3"

training_args_iter3 = TrainingArguments(
    output_dir=output_dir_iter3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="no",
    bf16=False,
    report_to="none"
)

trainer_iter3 = SFTTrainer(
    model=model,
    train_dataset=sft_dataset_iter3,
    processing_class=tokenizer,
    args=training_args_iter3
)

print("Iter3 LoRA SFT eğitimini başlatıyoruz...")
trainer_iter3.train()

trainer_iter3.model.save_pretrained(output_dir_iter3)
tokenizer.save_pretrained(output_dir_iter3)
print("Eğitilmiş LoRA iter3 model kaydedildi:", output_dir_iter3)

# Final model
model = trainer_iter3.model


In [ ]:

print("=== ITERATION 3 (FINAL) TEST SONUÇLARI ===")

iter3_eval_self = evaluate_model_on_dataset(
    model,
    tokenizer,
    test_500,
    use_self_improve=True,
    max_samples=50,
    desc="Iter3 - Self-improve"
)

iter3_eval_draft = evaluate_model_on_dataset(
    model,
    tokenizer,
    test_500,
    use_self_improve=False,
    max_samples=50,
    desc="Iter3 - Draft only"
)

print("\nIter3 Self-improve:", iter3_eval_self)
print("Iter3 Draft only  :", iter3_eval_draft)
